In [ ]:
pip install scikit-surprise

In [3]:
import pandas as pd
from surprise import SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate, GridSearchCV

In [ ]:
# Завантаження датасету MovieLens
data = Dataset.load_builtin('ml-100k')

# Параметри для пошуку за сіткою
param_grid_svd = {
    'n_factors': [50, 100, 150],
    'n_epochs': [20, 30, 40],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.02, 0.1, 0.2]
}

param_grid_svdpp = {
    'n_factors': [50, 100, 150],
    'n_epochs': [20, 30, 40],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.02, 0.1, 0.2]
}

param_grid_nmf = {
    'n_factors': [50, 100, 150],
    'n_epochs': [20, 30, 40],
    'reg_pu': [0.02, 0.1, 0.2],
    'reg_qi': [0.02, 0.1, 0.2]
}


In [5]:
# Створення екземплярів GridSearchCV для кожного алгоритму
gs_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=5, n_jobs=-1)
gs_svdpp = GridSearchCV(SVDpp, param_grid_svdpp, measures=['rmse', 'mae'], cv=5, n_jobs=-1)
gs_nmf = GridSearchCV(NMF, param_grid_nmf, measures=['rmse', 'mae'], cv=5, n_jobs=-1)


In [6]:
gs_svd.fit(data)

In [ ]:
gs_svdpp.fit(data)

In [8]:
gs_nmf.fit(data)

In [9]:
# Найкращі параметри для кожного алгоритму
best_params_svd = gs_svd.best_params['rmse']
#best_params_svdpp = gs_svdpp.best_params['rmse']
best_params_nmf = gs_nmf.best_params['rmse']

In [10]:
print("Найкращі параметри для SVD:")
print(best_params_svd)
#print("Найкращі параметри для SVD++:")
#print(best_params_svdpp)
print("Найкращі параметри для NMF:")
print(best_params_nmf)

Найкращі параметри для SVD:
{'n_factors': 150, 'n_epochs': 40, 'lr_all': 0.01, 'reg_all': 0.1}
Найкращі параметри для NMF:
{'n_factors': 150, 'n_epochs': 40, 'reg_pu': 0.2, 'reg_qi': 0.1}


In [14]:
# Найкращі моделі з найкращими параметрами
best_svd = gs_svd.best_estimator['rmse']

In [ ]:
best_svdpp = gs_svdpp.best_estimator['rmse']

In [16]:
best_nmf = gs_nmf.best_estimator['rmse']

In [17]:
# Проведення остаточної крос-валідації на найкращих моделях
results_svd = cross_validate(best_svd, data, measures=['rmse', 'mae'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9101  0.9014  0.9041  0.9150  0.9165  0.9094  0.0059  
MAE (testset)     0.7185  0.7135  0.7159  0.7219  0.7247  0.7189  0.0040  
Fit time          3.13    3.93    3.09    3.09    3.42    3.33    0.32    
Test time         0.11    0.12    0.12    0.12    0.41    0.18    0.12    


In [ ]:
results_svdpp = cross_validate(best_svdpp, data, measures=['rmse', 'mae'], cv=5, verbose=True)

In [18]:
results_nmf = cross_validate(best_nmf, data, measures=['rmse', 'mae'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9456  0.9365  0.9417  0.9338  0.9475  0.9410  0.0052  
MAE (testset)     0.7474  0.7394  0.7409  0.7373  0.7471  0.7424  0.0041  
Fit time          4.82    5.08    5.39    4.77    5.82    5.18    0.39    
Test time         0.10    0.18    0.27    0.10    0.13    0.16    0.07    


In [19]:
print("Результати крос-валідації для SVD:")
print(results_svd)
#print("Результати крос-валідації для SVD++:")
#print(results_svdpp)
print("Результати крос-валідації для NMF:")
print(results_nmf)

Результати крос-валідації для SVD:
{'test_rmse': array([0.91006137, 0.90140209, 0.90411628, 0.91497867, 0.91654639]), 'test_mae': array([0.7185473 , 0.71352669, 0.71593733, 0.72190048, 0.72469947]), 'fit_time': (3.1345205307006836, 3.9330787658691406, 3.092557191848755, 3.085883855819702, 3.41977858543396), 'test_time': (0.11170077323913574, 0.11947298049926758, 0.1240537166595459, 0.12439155578613281, 0.4085547924041748)}
Результати крос-валідації для NMF:
{'test_rmse': array([0.94564369, 0.93647382, 0.94167508, 0.93376964, 0.94747534]), 'test_mae': array([0.74742049, 0.7394465 , 0.74093347, 0.73730139, 0.74706894]), 'fit_time': (4.815887451171875, 5.075135946273804, 5.3919761180877686, 4.770459413528442, 5.824940919876099), 'test_time': (0.09752726554870605, 0.17903923988342285, 0.273608922958374, 0.10020756721496582, 0.12866449356079102)}


In [20]:
# Вибір найкращого алгоритму
best_algorithm = min([('SVD', results_svd), ('NMF', results_nmf)], key=lambda x: x[1]['test_rmse'].mean())
print(f"Найкращий алгоритм: {best_algorithm[0]} з середнім RMSE: {best_algorithm[1]['test_rmse'].mean()}")

Найкращий алгоритм: SVD з середнім RMSE: 0.9094209609047968
